In [1]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
import matplotlib.pyplot as plt
%matplotlib notebook
import s2fft
from croissant import crojax

In [2]:
# simple beam in topocentric coordinates
lmax = 8
freq = jnp.linspace(40, 80, 41)
beam_alm = crojax.Alm.zeros(lmax, frequencies=freq, coord="T")

# set (l=0, m=0) and (l=1, m=0) mode
beam_alm[:, 0, 0] = 30 * (freq/freq[0]) ** 2
beam_alm[:, 1, 0] = 10 * (freq/freq[0])**2

# visualize with healpix
#nside = 64
#hp.mollview(beam.hp_map(nside)[0], title=f"Beam at {freq[0]:.0f} MHz")

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
beam_alm.alm[:, 0, 0  # WHYYYYYYYYYYYYYYYYYYYyy

Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float64)

In [ ]:
# we can impose a horizon like this, note that the sharp edge creates ripples since we don't have an inifinite lmax
beam.horizon_cut()
hp.mollview(beam.hp_map(nside)[0], title=f"Beam at {freq[0]:.0f} MHz")

We use the Global Sky Model (Zheng et al 2016) at 25 MHz as the sky model. It has a built-in interface in the sky module of croissant.

In [ ]:
sky = cro.Sky.gsm(beam.frequencies, lmax=beam.lmax)
hp.mollview(sky.hp_map(nside)[0], title=f"Sky at {freq[0]:.0f} MHz")

In [ ]:
plt.figure()
plt.plot(sky.frequencies, np.real(sky[:, 0, 0]), label="Sky monopole spectrum")
plt.xlabel("Frequency [MHz]")
plt.ylabel("Temperature [K]")
plt.xlim(sky.frequencies.min(), sky.frequencies.max())
plt.legend()
plt.show()

In [ ]:
# let's do a full sidereal day on the moon
loc = (20., -10.)
t_start = Time("2022-06-02 15:43:43")
t_end = t_start + cro.constants.sidereal_day_moon * seconds
sim = cro.Simulator(beam, sky, loc, t_start, world="moon", t_end=t_end, N_times=300, lmax=lmax)

In [ ]:
# the simulator view of the beam and sky after moving to MCMF coordinates
hp.mollview(sim.beam.hp_map(nside)[0], title=f"Beam at {freq[0]:.0f} MHz")
hp.mollview(sim.sky.hp_map(nside)[0], title=f"Sky at {freq[0]:.0f} MHz")

Run the simulator!

In [ ]:
# dpss mode
sim.run(dpss=True, nterms=40)
sim.plot(power=2.5)

In [ ]:
sim.run(dpss=False)
sim.plot(power=2.5)

In [ ]:
plt.figure()
plt.plot(sim.frequencies, sim.waterfall[::10].T, ls="--")
plt.xlim(sim.frequencies.min(), sim.frequencies.max())
plt.xlabel("$\\nu$ [MHz]")
plt.ylabel("Temperature [K]")
plt.grid()
plt.show()

In [ ]:
# Temp vs time
fig, axs = plt.subplots(figsize=(13,5), ncols=5, sharex=True, sharey=True)
for i, f in enumerate(sim.frequencies[::10]):
    ax = axs.ravel()[i]
    fidx = np.argwhere(sim.frequencies == f)[0, 0]
    ax.plot(sim.waterfall[:, fidx] * f**2.5)
    ax.set_title(f"{f} MHz")
    ax.grid()
plt.tight_layout()
plt.show()

# On Earth

In [ ]:
loc = (20., -10.)
t_start = Time("2022-06-02 15:43:43")
t_end = t_start + cro.constants.sidereal_day_earth * seconds
sim = cro.Simulator(beam, sky, loc, t_start, world="earth", t_end=t_end, N_times=300, lmax=lmax)

In [ ]:
# the simulator view of the beam and sky after moving to equatorial coordinates
hp.mollview(sim.beam.hp_map(nside)[0], title=f"Beam at {freq[0]:.0f} MHz")
hp.mollview(sim.sky.hp_map(nside)[0], title=f"Sky at {freq[0]:.0f} MHz")

In [ ]:
# dpss mode
sim.run(dpss=True, nterms=40)
sim.plot(power=2.5)

In [ ]:
# Temp vs time
fig, axs = plt.subplots(figsize=(13,5), ncols=5, sharex=True, sharey=True)
for i, f in enumerate(sim.frequencies[::10]):
    ax = axs.ravel()[i]
    fidx = np.argwhere(sim.frequencies == f)[0, 0]
    ax.plot(sim.waterfall[:, fidx] * f**2.5)
    ax.set_title(f"{f} MHz")
    ax.grid()
plt.tight_layout()
plt.show()